In [1]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

from datetime import datetime

import pickle
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops_zoomcamp_project/02-experiment-tracking/mlruns/2', creation_time=1720022487068, experiment_id='2', last_update_time=1720022487068, lifecycle_stage='active', name='mlops_zoomcamp_experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops_zoomcamp_project/02-experiment-tracking/mlruns/1', creation_time=1720020560374, experiment_id='1', last_update_time=1720020560374, lifecycle_stage='active', name='student_performance-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops_zoomcamp_project/02-experiment-tracking/mlruns/0', creation_time=1720020560347, experiment_id='0', last_update_time=1720020560347, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.test_accuracy > 0.69",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.test_accuracy DESC"]
)


for run in runs:
    print(f"run id: {run.info.run_id}, test_accuracy: {run.data.metrics['test_accuracy']:.4f}")

run id: e6b8b8c4e9864995ac5be24d398a6ec4, test_accuracy: 0.7453
run id: d41515113bce483e85759ee94d6c0b51, test_accuracy: 0.7453
run id: 4528febfbe6943a5b35f42785f292325, test_accuracy: 0.7328
run id: 0f1a32ee243948dbbc79596c0318307c, test_accuracy: 0.6973


In [11]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

for run in runs[::-1]:
    run_id = run.info.run_id
    model_uri = f"runs:/{run_id}/model"
    mlflow.register_model(model_uri=model_uri, name="mlops-zoomcamp-registry")

Successfully registered model 'mlops-zoomcamp-registry'.
Created version '1' of model 'mlops-zoomcamp-registry'.
Registered model 'mlops-zoomcamp-registry' already exists. Creating a new version of this model...
Created version '2' of model 'mlops-zoomcamp-registry'.
Registered model 'mlops-zoomcamp-registry' already exists. Creating a new version of this model...
Created version '3' of model 'mlops-zoomcamp-registry'.
Registered model 'mlops-zoomcamp-registry' already exists. Creating a new version of this model...
Created version '4' of model 'mlops-zoomcamp-registry'.


In [13]:
model_version = 1
new_stage = "Archived"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_33889/601725576.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1720023150511, current_stage='Archived', description=None, last_updated_timestamp=1720023188429, name='mlops-zoomcamp-registry', run_id='0f1a32ee243948dbbc79596c0318307c', run_link=None, source='/workspaces/mlops_zoomcamp_project/02-experiment-tracking/mlruns/2/0f1a32ee243948dbbc79596c0318307c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [14]:
model_version = 2
new_stage = "None"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_33889/3423046592.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1720023150548, current_stage='None', description=None, last_updated_timestamp=1720023206363, name='mlops-zoomcamp-registry', run_id='4528febfbe6943a5b35f42785f292325', run_link=None, source='/workspaces/mlops_zoomcamp_project/02-experiment-tracking/mlruns/2/4528febfbe6943a5b35f42785f292325/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [20]:
new_stage = "Staging"
for model_version in [3,4]:
    client.transition_model_version_stage(
        name=model_name,
        version=model_version,
        stage=new_stage,
        archive_existing_versions=False
    )

/tmp/ipykernel_33889/3063246248.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [21]:
date = datetime.today().date()
new_stage = "Staging"
for model_version in [3,4]:
    client.update_model_version(
        name=model_name,
        version=model_version,
        description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
    )

In [22]:
model_name = "mlops-zoomcamp-registry"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 2, stage: None
version: 4, stage: Staging


/tmp/ipykernel_33889/2309309639.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [24]:
# load data
with open("/workspaces/mlops_zoomcamp_project/dumps/misc/preprocessed_data.pkl", "rb") as f:
    X_train, X_test, y_train, y_test = pickle.load(f)

# load model
model_version = 4
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(model)
print({"train_accuracy": accuracy_score(y_train.values, y_train_pred)})
print({"test_accuracy": accuracy_score(y_test.values, y_test_pred)})

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: e6b8b8c4e9864995ac5be24d398a6ec4

{'train_accuracy': 0.7684265551489806}
{'test_accuracy': 0.7453027139874739}


In [25]:
model_version = 4
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

/tmp/ipykernel_33889/3194717379.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1720023150631, current_stage='Production', description='The model version 4 was transitioned to Production on 2024-07-03', last_updated_timestamp=1720023433040, name='mlops-zoomcamp-registry', run_id='e6b8b8c4e9864995ac5be24d398a6ec4', run_link=None, source='/workspaces/mlops_zoomcamp_project/02-experiment-tracking/mlruns/2/e6b8b8c4e9864995ac5be24d398a6ec4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>